In [4]:
from nba_api.stats.endpoints import playergamelogs

In [7]:
log = playergamelogs.PlayerGameLogs()
df = log.get_data_frames()[0]
print(df.head())    

Error occurred: ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

Trying alternative approach...
Alternative approach also failed: ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
The NBA API might be temporarily unavailable or rate limiting requests.
Alternative approach also failed: ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
The NBA API might be temporarily unavailable or rate limiting requests.


In [8]:
# More robust approach with better error handling and retries
def get_player_game_logs_safely(max_retries=3, delay=5):
    """
    Safely fetch player game logs with retries and proper error handling
    """
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt + 1}/{max_retries}...")
            
            # Create headers that mimic a real browser
            headers = {
                'Host': 'stats.nba.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate, br',
                'x-nba-stats-origin': 'stats',
                'x-nba-stats-token': 'true',
                'Connection': 'keep-alive',
                'Referer': 'https://stats.nba.com/',
                'Pragma': 'no-cache',
                'Cache-Control': 'no-cache',
            }
            
            # Add delay between attempts
            if attempt > 0:
                print(f"Waiting {delay} seconds before retry...")
                time.sleep(delay)
            else:
                time.sleep(2)  # Always wait a bit initially
            
            # Try to get specific season data (less load on API)
            log = playergamelogs.PlayerGameLogs(
                season_nullable='2023-24',
                season_type_nullable='Regular Season',
                headers=headers,
                timeout=60  # Increase timeout
            )
            
            df = log.get_data_frames()[0]
            print(f"✅ Success! Retrieved {len(df)} game logs")
            return df
            
        except requests.exceptions.Timeout:
            print(f"⏰ Timeout on attempt {attempt + 1}")
            if attempt == max_retries - 1:
                print("❌ All attempts failed due to timeout")
                return None
                
        except Exception as e:
            print(f"❌ Error on attempt {attempt + 1}: {type(e).__name__}: {str(e)}")
            if attempt == max_retries - 1:
                print("❌ All attempts failed")
                return None
    
    return None

# Try to get the data
df = get_player_game_logs_safely()

if df is not None:
    print("\nFirst few rows:")
    print(df.head())
    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
else:
    print("\n⚠️  Unable to fetch data from NBA API.")
    print("This is common and can be due to:")
    print("1. NBA API rate limiting")
    print("2. Server maintenance")
    print("3. Network issues")
    print("\n💡 Try again in a few minutes or use cached data if available.")

Attempt 1/3...
✅ Success! Retrieved 42926 game logs

First few rows:
  SEASON_YEAR  PLAYER_ID       PLAYER_NAME NICKNAME     TEAM_ID  \
0     2023-24     2544.0      LeBron James   LeBron  1610612747   
1     2023-24  1641713.0        GG Jackson       GG  1610612763   
2     2023-24  1630202.0  Payton Pritchard   Payton  1610612738   
3     2023-24   203078.0      Bradley Beal  Bradley  1610612756   
4     2023-24  1628973.0     Jalen Brunson    Jalen  1610612752   

  TEAM_ABBREVIATION           TEAM_NAME     GAME_ID            GAME_DATE  \
0               LAL  Los Angeles Lakers  0022301195  2024-04-14T00:00:00   
1               MEM   Memphis Grizzlies  0022301193  2024-04-14T00:00:00   
2               BOS      Boston Celtics  0022301186  2024-04-14T00:00:00   
3               PHX        Phoenix Suns  0022301194  2024-04-14T00:00:00   
4               NYK     New York Knicks  0022301190  2024-04-14T00:00:00   

       MATCHUP  ... PFD_RANK  PTS_RANK  PLUS_MINUS_RANK  NBA_FANTASY_PT